Project of Advanced Algorithms and Parallel of Programming by Daniele Asciutti.
2022/2023

In [ ]:
!apt install clang-10
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  binfmt-support libclang-common-10-dev libclang-cpp10 libclang1-10 libffi-dev
  libomp-10-dev libomp5-10 libpfm4 libz3-4 libz3-dev llvm-10 llvm-10-dev
  llvm-10-runtime llvm-10-tools python3-pkg-resources python3-pygments
  python3-yaml
Suggested packages:
  clang-10-doc libomp-10-doc llvm-10-doc python3-setuptools ttf-bitstream-vera
The following NEW packages will be installed:
  binfmt-support clang-10 libclang-common-10-dev libclang-cpp10 libclang1-10
  libffi-dev libomp-10-dev libomp5-10 libpfm4 libz3-4 libz3-dev llvm-10
  llvm-10-dev llvm-10-runtime llvm-10-tools python3-pkg-resources
  python3-pygments python3-yaml
0 upgraded, 18 newly installed, 0 to remove and 22 not upgraded.
Need to get 61.2 MB o

In [ ]:
!cd benchmark

In [ ]:
%%writefile prog.cc

#include <benchmark/benchmark.h>

#include <iostream>
#include <vector>
#include <random>
#include <algorithm>



typedef struct {
    int firstpos;
    int secondpos;
}wall_t;

int R;
int C;

//It creates all the different sets
void createSets(int nodes[]){
    for(int i=0;i<R*C;i++) {
        nodes[i]=i;
    }
}

//It creates all the walls of the maze
std::vector<wall_t> createWalls(){
    std::vector<wall_t> walls;
    wall_t tmpwall;
    for(int i=0;i<R*C-1;i++){
        if(i%C!=C-1) {
            tmpwall.firstpos = i;
            tmpwall.secondpos = i + 1;
            walls.push_back(tmpwall);
        }
        if(i<C*R-C){
            tmpwall.firstpos = i;
            tmpwall.secondpos = i + C;
            walls.push_back(tmpwall);
        }
    }
    return walls;
}

//It takes a set and changes the tree of the set to have deep equals to 1
void mergechildren(int pos,int nodes[]){
    int parent=pos;
    while(parent!=nodes[parent]){
        parent=nodes[parent];
    }
    int tmp;
    while(nodes[pos]!=parent){
        tmp=nodes[pos];
        nodes[pos]=parent;
        pos=tmp;
    }

}

//It takes two different position and checks if one can reach the other
bool find(int pos1,int pos2,int nodes[]){
    mergechildren(pos1,nodes);
    mergechildren(pos2,nodes);

    return nodes[pos1]==nodes[pos2];
}

//It takes the positions of two different sets and unites them
void setsUnion(int pos1,int pos2,int nodes[]){
    int tmp=nodes[pos2];
    nodes[tmp]=nodes[pos1];
}

//It takes a empty array of int and creates the maze
void randMaze(int nodes[]){
    createSets(nodes);
    std::vector<wall_t> walls=createWalls();
    int start=0;
    int end=R*C-1;
    std::random_device  rd;
    auto rng = std::default_random_engine { rd()};
    std::shuffle(std::begin(walls), std::end(walls), rng);
    int c=0;
    wall_t tmpwall;
    while(!find(start,end,nodes)){
        tmpwall = walls[c];
       c++;
        if(!find(tmpwall.firstpos,tmpwall.secondpos,nodes)){
            setsUnion(tmpwall.firstpos,tmpwall.secondpos,nodes);
        }
    }
}



//Main of the program (called in this way to let benchmark recognize it)
int prog(int row,int col){
    R=row;
    C=col;
    int nodes[R*C];

    randMaze(nodes);


}



static void BM_test(benchmark::State& state){
  for (auto _ : state) {
    benchmark::DoNotOptimize(prog(state.range(0),state.range(0)));
  }
  state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_test)
  ->RangeMultiplier(2)->Range(1, 512)->Complexity();

BENCHMARK_MAIN();


Overwriting prog.cc


In [ ]:
!g++ prog.cc -std=c++11 -isystem benchmark/include \-Lbenchmark/build/src -lbenchmark -lpthread -o prog

In [ ]:
!./prog

2022-10-19T21:26:15+00:00
Running ./prog
Run on (2 X 2200 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.09, 0.06, 0.08
------------------------------------------------------
Benchmark            Time             CPU   Iterations
------------------------------------------------------
BM_test/1         2116 ns         2082 ns       341284
BM_test/2         3428 ns         3401 ns       205118
BM_test/4         7479 ns         7434 ns        98682
BM_test/8        20986 ns        20962 ns        33339
BM_test/16       75259 ns        75159 ns         8417
BM_test/32      298753 ns       298112 ns         2360
BM_test/64     1201397 ns      1198701 ns          580
BM_test/128    5092956 ns      5085923 ns          100
BM_test/256   23736396 ns     23694909 ns           31
BM_test/512  120334643 ns    120167448 ns            6
BM_test_BigO     452.78 N^2      452.15 N^2  
BM_test_RMS         14

In [ ]:
%%writefile prog.cc

#include <iostream>
#include <vector>
#include <random>
#include <algorithm>

#define R 5
#define C 5


typedef struct {
    int firstpos;
    int secondpos;
}wall_t;

void createSets(int nodes[]){
    for(int i=0;i<R*C;i++) {
        nodes[i]=i;
    }
}


std::vector<wall_t> createWalls(){
    std::vector<wall_t> walls;
    wall_t tmpwall;
    for(int i=0;i<R*C-1;i++){
        if(i%C!=C-1) {
            tmpwall.firstpos = i;
            tmpwall.secondpos = i + 1;
            walls.push_back(tmpwall);
        }
        if(i<C*R-C){
            tmpwall.firstpos = i;
            tmpwall.secondpos = i + C;
            walls.push_back(tmpwall);
        }
    }
    return walls;
}

void mergechildren(int pos,int nodes[]){
    int parent=pos;
    while(parent!=nodes[parent]){
        parent=nodes[parent];
    }
    int tmp;
    while(nodes[pos]!=parent){
        tmp=nodes[pos];
        nodes[pos]=parent;
        pos=tmp;
    }

}

bool find(int pos1,int pos2,int nodes[]){
    mergechildren(pos1,nodes);
    mergechildren(pos2,nodes);

    return nodes[pos1]==nodes[pos2];
}


void setsUnion(int pos1,int pos2,int nodes[]){
    int tmp=nodes[pos2];
    nodes[tmp]=nodes[pos1];
}

//It takes an empty matrix and fill it with the maze
std::vector<wall_t> randMaze(int nodes[]){
    createSets(nodes);
    std::vector<wall_t> walls=createWalls();
    std::vector<wall_t> wallsInMaze;
    int start=0;
    int end=R*C-1;
    std::random_device  rd;
    auto rng = std::default_random_engine { rd()};
    std::shuffle(std::begin(walls), std::end(walls), rng);
    for(int i=0;i<walls.size();i++)
        wallsInMaze.push_back(walls[i]);
    int c=0;
    wall_t tmpwall;
    while(!find(start,end,nodes)){
        tmpwall = walls[c];
        c++;
        if(!find(tmpwall.firstpos,tmpwall.secondpos,nodes)){
            wallsInMaze.erase(walls.begin()+c);
            setsUnion(tmpwall.firstpos,tmpwall.secondpos,nodes);
        }
    }
    return wallsInMaze;
}

//It's used to check if the maze is been created well. It takes the walls not eliminated from the maze and
//  checks that there is a path from the start to the end using a depth first algorithm
void testMaze(std::vector<wall_t> wallsInMaze){
    int graph[R*R][C*C];
    for(int i=0;i<R*R;i++)
        for(int j=0;j<C*C;j++){
            if(i==j || i==j+1 || i==j-1 || i==j+C || i==j-C)
                graph[i][j]=1;
            else graph[i][j]=0;
        }

    int visitedNodes[R*C];
    for(int i=0;i<R*C;i++)
        visitedNodes[i]=0;

    for(wall_t tmp: wallsInMaze){
        graph[tmp.firstpos][tmp.secondpos]=0;
        graph[tmp.secondpos][tmp.firstpos]=0;
    }

    int i=0;
    visitedNodes[i]=1;
    int k=2;
    while(i!=R*R-1){
        bool check = false;
        for(int j=0; j<C*R;j++)
            if(graph[i][j]==1 && visitedNodes[j]==0){
                check=true;
                graph[i][j]=0;
                graph[j][i]=0;
                i=j;
                visitedNodes[j]=k;
                k++;
            }
        if(!check){
            for(int l=0;l<C*R;l++) {
                if (visitedNodes[l] == k - 1)
                    visitedNodes[l] = 0;
                else if (visitedNodes[l] == k - 2)
                    i = l;
            }
            k--;
        }
    }

    std::cout<<"\nCorrect!";
}


int main(){
    int nodes[R*C];

    std::vector<wall_t> wallsInMaze=randMaze(nodes);

    for(int i=0;i<R*C;i++)
        mergechildren(i,nodes);
    std::cout<<"\n";
    for(int i=0;i<R*C;i++)
        if(i%C!=C-1)
            std::cout<<nodes[i]<<" ";
        else std::cout<<nodes[i]<<"\n";

        testMaze(wallsInMaze);

}


Overwriting prog.cc


In [ ]:
!g++ prog.cc -std=c++11 -isystem benchmark/include \-Lbenchmark/build/src -lbenchmark -lpthread -o prog

In [ ]:
!./prog


0 0 0 0 0
0 6 0 0 0
6 6 0 0 14
0 0 0 0 19
0 0 0 0 0

Correct!